# Challenge

In [1]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import chi2_contingency
from scipy import interpolate
from scipy import stats
from sklearn.linear_model import LinearRegression
import math
from copy import deepcopy

## Challenge 1 Combination Problem Version 1

### Use recursion / dynamic programming to find all path

In [2]:
x = 2
y = 1

### Tried memoization, but only return the first path, need all the path

In [3]:
import collections
import functools
class memoized(object):
    def __init__(self, func):
        self.func = func
        self.cache = {}
    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value
    def __repr__(self):
        return self.func.__doc__
    def __get__(self, obj, objtype):
        return functools.partial(self.__call__, obj)

In [4]:
allx = []
mx = []
def recursion(x, y):
    global mx
    global allx
    mx.extend([[x, y]])
    if x == 0 and y == 0:
        allx.extend([mx])
        mx = []
        return 0
    elif x != 0 and y == 0:
        return recursion(x - 1, y)
    elif x == 0 and y != 0:
        return recursion(x, y - 1)
    else:
        return recursion(x - 1, y) + recursion(x, y - 1)

In [5]:
recursion(x, y)

0

### Construct the list for all path points

In [6]:
allx

[[[2, 1], [1, 1], [0, 1], [0, 0]], [[1, 0], [0, 0]], [[2, 0], [1, 0], [0, 0]]]

In [7]:
alyx = deepcopy(allx)

In [8]:
for i in range(1, len(allx)):
    while len(alyx[i]) != len(allx[0]):
        j = i
        while len(alyx[i]) >= len(allx[j-1]):
            j = j - 1
        alyx[i].extend(allx[j-1][0:(len(allx[j-1]) - len(alyx[i]))])

In [9]:
allx

[[[2, 1], [1, 1], [0, 1], [0, 0]], [[1, 0], [0, 0]], [[2, 0], [1, 0], [0, 0]]]

In [10]:
alyx

[[[2, 1], [1, 1], [0, 1], [0, 0]],
 [[1, 0], [0, 0], [2, 1], [1, 1]],
 [[2, 0], [1, 0], [0, 0], [2, 1]]]

### Finding probability for each path

In [11]:
def probability(x, y, alyx):
    p = []
    for i in alyx:
        count = 0
        for j in i:
            if j[0] < x and j[1] < y:
                count += 1
        prob = (1/2)**count
        p.append(prob)
    return p

In [12]:
probability(x, y, alyx)

[0.5, 0.25, 0.25]

### Finding deviation for each path

My first assumption, which assumes that deviation is the sum of all the point, the next assumption which assumes that deviation is the max is what I used ine the end.  So disregard this function.
def deviation(x, y, alyx):
    d = []
    for i in alyx:
        dev = 0
        for j in i:
            dev += abs(j[0]/x - j[1]/y)
        d.append(dev)
    return d

In [13]:
def deviation2(x, y, alyx):
    d = []
    for i in alyx:
        dev = 0
        for j in i:
            if abs(j[0]/x - j[1]/y) > dev:
                dev = abs(j[0]/x - j[1]/y)
        d.append(dev)
    return d

In [14]:
deviation2(x, y, alyx)

[1.0, 0.5, 1.0]

### Finding Expected Mean

In [15]:
def dmean(x, y, alyx):
    return (np.array(deviation2(x, y, alyx))*np.array(probability(x, y, alyx))).sum()

In [16]:
dmean(x, y, alyx)

0.875

### Finding STD for probability

In [17]:
def stdp(x, y, alyx):
    return np.sqrt((np.square(np.array(deviation2(x, y, alyx)))*np.array(probability(x, y, alyx))).sum() - np.square(dmean(x, y, alyx)))

In [18]:
stdp(x, y, alyx)

0.21650635094610965

### Conditional Probability

In [19]:
def cp(x, y, alyx):
    return (np.array(deviation2(x, y, alyx)) > 0.6).sum()/(np.array(deviation2(x, y, alyx)) > 0.2).sum()

In [20]:
cp(x, y, alyx)

0.66666666666666663

## Challenge 1 Another Assumption

In [107]:
x = 2
y = 1
def allpoint(x, y):
    point = []
    for i in range(x+1):
        for j in range(y+1):
            point.append([i,j])
    return point

In [108]:
point = allpoint(2, 1)

In [109]:
path = []
for i in point:
    for j in point:
        if j[0] == i[0] - 1 and j[1] == i[1]:
            path.append([j, i])
        if j[0] == i[0] and j[1] == i[1] - 1:
            path.append([j, i])

In [110]:
point

[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0], [2, 1]]

In [111]:
path

[[[0, 0], [0, 1]],
 [[0, 0], [1, 0]],
 [[0, 1], [1, 1]],
 [[1, 0], [1, 1]],
 [[1, 0], [2, 0]],
 [[1, 1], [2, 1]],
 [[2, 0], [2, 1]]]

In [112]:
deviation_p = []
for i in path:
    d = 0
    for j in i:
        if abs(j[0]/x - j[1]/y) > d:
            d = abs(j[0]/x - j[1]/y)
    deviation_p.append(d)

In [113]:
deviation_p

[1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0]

### wrong assumption

In [117]:
prob_p = []
for i in path:
    if i[1][0] < x and i[1][1] < y:
        c = i[1][0] + i[1][1]
    elif i[1][0] < x and i[1][1] == y:
        
    p = 1/2**c
    prob_p.append(p)

In [118]:
prob_p

[0.5, 0.5, 0.25, 0.25, 0.25, 0.125, 0.125]

## Challenge 2

In [2]:
MT = pd.read_csv(r'C:\Users\zefan\Desktop\CapStone_Project\data\MT_cleaned.csv')
VT = pd.read_csv(r'C:\Users\zefan\Desktop\CapStone_Project\data\VT_cleaned.csv')

C:\Users\zefan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\zefan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
MT.dtypes

id                        object
state                     object
stop_date                 object
stop_time                 object
location_raw              object
county_name               object
county_fips              float64
fine_grained_location     object
police_department        float64
driver_gender             object
driver_age_raw           float64
driver_age               float64
driver_race_raw           object
driver_race               object
violation_raw             object
violation                 object
search_conducted            bool
search_type_raw           object
search_type               object
contraband_found          object
stop_outcome              object
is_arrested                 bool
lat                      float64
lon                      float64
ethnicity                 object
city                      object
out_of_state              object
vehicle_year              object
vehicle_make              object
vehicle_model             object
vehicle_st

In [4]:
VT.dtypes

id                        object
state                     object
stop_date                 object
stop_time                 object
location_raw              object
county_name               object
county_fips              float64
fine_grained_location     object
police_department         object
driver_gender             object
driver_age_raw           float64
driver_age               float64
driver_race_raw           object
driver_race               object
violation_raw             object
violation                 object
search_conducted            bool
search_type_raw           object
search_type               object
contraband_found          object
stop_outcome              object
is_arrested                 bool
officer_id               float64
dtype: object

In [5]:
MT.head(5)

,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,lon,ethnicity,city,out_of_state,vehicle_year,vehicle_make,vehicle_model,vehicle_style,search_reason,stop_outcome_raw
0,MT-2009-00001,MT,2009-01-01,02:10,CASCADE,Cascade County,30013.0,US 89 N MM10 (SB),NaN,F,...,-111.802932,N,NaN,False,1994,FORD,EXPLORER,SPORT UTILITY,NaN,"TRAFFIC CITATION,WARNING"
1,MT-2009-00002,MT,2009-01-02,11:34,MISSOULA,Missoula County,30063.0,HWY 93 SO AND ANNS LANE S/B,NaN,M,...,-114.081142,N,NaN,False,1996,GMC,TK,TRUCK,NaN,"INFFRACTION ARREST,WARNING"
2,MT-2009-00003,MT,2009-01-03,11:36,MISSOULA,Missoula County,30063.0,P007 HWY 93 MM 77 N/B,NaN,M,...,-114.073505,N,NaN,False,1999,GMC,YUKON,SPORT UTILITY,NaN,INFFRACTION ARREST
3,MT-2009-00004,MT,2009-01-04,10:33,MISSOULA,Missoula County,30063.0,P007 HWY 93 MM 81 S/B,NaN,F,...,-114.079027,NaN,NaN,False,2002,HOND,CR-V,SPORT UTILITY,NaN,INFFRACTION ARREST
4,MT-2009-00005,MT,2009-01-04,10:46,MISSOULA,Missoula County,30063.0,P007 HWY 93 MM 81 N/B,NaN,M,...,-114.079150,NaN,NaN,False,1992,TOYT,TERCEL,SEDAN,NaN,INFFRACTION ARREST


In [6]:
MT.shape

(825118, 33)

### Q1 contains missing values, use the total number of stops as denominator

In [7]:
MT['driver_gender'].value_counts()

M    556934
F    268065
Name: driver_gender, dtype: int64

In [8]:
MT[MT['driver_gender'].isnull() == True].shape

(119, 33)

In [9]:
MT[MT['driver_gender'] == "M"]['id'].count()/MT['id'].count()

0.67497497327654954

### Q2 contains mising values, arrest more likely compare to in-state arrest, so use in state arrest as denominator

In [10]:
MT['out_of_state'].value_counts()

False    616778
True     203641
Name: out_of_state, dtype: int64

In [11]:
MT[MT['out_of_state'].isnull() == True].shape

(4699, 33)

In [12]:
MT['stop_outcome'].value_counts()

Warning                    422671
Citation                   383686
Arrest                      17195
Faulty Equipment Notice      1316
No Action                     197
Name: stop_outcome, dtype: int64

In [13]:
MT[MT['stop_outcome'].isnull() == True].shape

(53, 33)

In [14]:
MT[(MT['out_of_state'] == True) & (MT['stop_outcome'] == 'Arrest')]['id'].count()/MT[(MT['out_of_state'] == False) & (MT['stop_outcome'] == 'Arrest')]['id'].count()

0.39934372436423299

In [15]:
osa = MT[(MT['out_of_state'] == True) & (MT['stop_outcome'] == 'Arrest')]['id'].count()
os = MT[MT['out_of_state'] == True]['id'].count()
isa = MT[(MT['out_of_state'] == False) & (MT['stop_outcome'] == 'Arrest')]['id'].count()
ins = MT[MT['out_of_state'] == False]['id'].count()

In [16]:
(osa/os)/(isa/ins)

1.2095129351452942

### Q3 out-of-state arrest, out-of-state total, in-state arret, in-state total

In [17]:
osa = MT[(MT['out_of_state'] == True) & (MT['stop_outcome'] == 'Arrest')]['id'].count()

In [18]:
os = MT[MT['out_of_state'] == True]['id'].count()

In [19]:
isa = MT[(MT['out_of_state'] == False) & (MT['stop_outcome'] == 'Arrest')]['id'].count()

In [20]:
ins = MT[MT['out_of_state'] == False]['id'].count()

#### Test for proportion equality

In [21]:
count = np.array([osa, isa])

In [22]:
nobs = np.array([os, ins])

In [23]:
proportions_chisquare(count, nobs)

(272349552.64835417, 0.0, (array([[  4868, 198773],
         [ 12190, 604588]]), array([[-1001.0240109 ,   473.51432987],
         [ 2353.75403544,  1918.71692148]])))

In [24]:
proportions_ztest(count, nobs)

(11.354843359336838, 7.0165534326186535e-30)

#### Test for independency

In [25]:
obs = np.array([[osa, isa], [os, ins]])

In [26]:
chi2, p, dof, expected = chi2_contingency(obs)

In [27]:
chi2

123.22977188345993

### Q4

In [28]:
MT.columns

Index(['id', 'state', 'stop_date', 'stop_time', 'location_raw', 'county_name',
       'county_fips', 'fine_grained_location', 'police_department',
       'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race_raw',
       'driver_race', 'violation_raw', 'violation', 'search_conducted',
       'search_type_raw', 'search_type', 'contraband_found', 'stop_outcome',
       'is_arrested', 'lat', 'lon', 'ethnicity', 'city', 'out_of_state',
       'vehicle_year', 'vehicle_make', 'vehicle_model', 'vehicle_style',
       'search_reason', 'stop_outcome_raw'],
      dtype='object')

In [29]:
MT[MT['violation'].str.contains('Speed', na = False)]['id'].count()/MT['id'].count()

0.65809981117852234

### Q5

In [30]:
MT_DUI = MT[MT['violation'].str.contains('DUI', na = False)]['id'].count()/MT['id'].count()

In [31]:
VT_DUI = VT[VT['violation'].str.contains('DUI', na = False)]['id'].count()/VT['id'].count()

In [32]:
MT_DUI/VT_DUI

4.0859996944208561

### Q6

In [33]:
MT2 = MT.copy()

In [34]:
MT2['stop_date'] = pd.to_datetime(MT2['stop_date'])
MT2['vehicle_year'] = pd.to_numeric(MT2['vehicle_year'], errors = 'coerce')

In [35]:
MT2['t_year'] = MT2['stop_date'].dt.year

In [36]:
extra = MT2.groupby('t_year')['vehicle_year'].mean()

In [37]:
f = interpolate.interp1d(extra.index, extra.values, kind = 'linear', fill_value = 'extrapolate')

In [38]:
f(2020)

array(2008.1775748284529)

In [39]:
extra

t_year
2009.0    2000.980215
2010.0    2001.521377
2011.0    2002.280938
2012.0    2003.362207
2013.0    2003.905175
2014.0    2004.482506
2015.0    2005.295767
2016.0    2005.872128
Name: vehicle_year, dtype: float64

### Q7

In [40]:
stats.ttest_ind(extra.index, extra.values)

Ttest_indResult(statistic=8.4710347229354035, pvalue=6.9923939171609433e-07)

In [41]:
stats.ttest_ind(extra.index, extra.values, equal_var = False)

Ttest_indResult(statistic=8.4710347229354035, pvalue=1.3902067899844079e-06)

In [42]:
l = stats.linregress(extra.index, extra.values)

In [43]:
l.pvalue

5.6091482537034642e-08

In [44]:
l

LinregressResult(slope=0.71741689995540237, intercept=559.66102804942648, rvalue=0.99717755450017109, pvalue=5.6091482537034642e-08, stderr=0.022051828385618037)

In [45]:
2020*l.slope + l.intercept

2008.8431659593393

In [46]:
x = np.array(extra.index.tolist()).reshape(-1, 1)

In [47]:
y = extra.values

In [48]:
lm = LinearRegression().fit(x, y)

In [49]:
lm.predict(2020)

array([ 2008.84316596])

### Q8

In [98]:
VMT = pd.concat([MT, VT], ignore_index = True)

In [99]:
VMT['stop_time'] = pd.to_datetime(VMT['stop_time'], format = '%H:%M').dt.hour

In [102]:
VMT['stop_time'].value_counts()

15.0    95891
16.0    86886
18.0    82430
14.0    82129
17.0    81437
8.0     62488
9.0     62233
10.0    61946
13.0    59281
19.0    57980
11.0    51008
20.0    47244
21.0    45891
22.0    44387
12.0    44024
7.0     41550
23.0    38599
0.0     25490
1.0     16856
6.0      8561
2.0      8399
5.0      1710
3.0      1425
4.0       547
Name: stop_time, dtype: int64

In [113]:
VMT.groupby('stop_time')['id'].count().sort_values()

stop_time
4.0       547
3.0      1425
5.0      1710
2.0      8399
6.0      8561
1.0     16856
0.0     25490
23.0    38599
7.0     41550
12.0    44024
22.0    44387
21.0    45891
20.0    47244
11.0    51008
19.0    57980
13.0    59281
10.0    61946
9.0     62233
8.0     62488
17.0    81437
14.0    82129
18.0    82430
16.0    86886
15.0    95891
Name: id, dtype: int64

In [114]:
95891 - 547

95344

### Q9

In [171]:
area = MT.groupby('county_name')['lon', 'lat'].agg('std')

In [172]:
area

,lon,lat
county_name,,
Beaverhead County,0.185625,0.268133
Big Horn County,1.937072,0.112396
Blaine County,0.258872,0.042086
Broadwater County,7.825885,3.248041
Carbon County,0.164160,0.150638
Carter County,0.218745,0.111636
Cascade County,3.512326,0.258816
Chouteau County,0.337009,0.179221
Custer County,0.187430,0.105790


In [173]:
area['ellipse'] = area['lon']*area['lat']*(math.pi)

In [174]:
area['lat_km'] = area['lat']*110.574

In [196]:
area['lon_km'] = (area['lat'].apply(math.cos))*111.320

In [201]:
area['area_km'] = abs(area['lat_km']*area['lon_km']*math.pi)

In [205]:
area.sort_values(by = 'area_km')

,lon,lat,ellipse,lat_km,lon_km,area_km
county_name,,,,,,
Blaine County,0.258872,0.042086,0.034227,4.653571,111.221430,1626.015602
Golden Valley County,0.162618,0.044875,0.022925,4.961956,111.207935,1733.558661
Treasure County,0.138295,0.045461,0.019751,5.026839,111.204986,1756.180464
Sweet Grass County,6.273852,0.058126,1.145661,6.427246,111.131997,2243.953787
Roosevelt County,0.539356,0.063369,0.107375,7.006959,111.096565,2445.569799
Wibaux County,0.081696,0.069986,0.017963,7.738686,111.047483,2699.764095
Hill County,0.213804,0.069994,0.047014,7.739475,111.047427,2700.037994
Prairie County,0.157117,0.071605,0.035344,7.917660,111.034737,2761.885144
Liberty County,0.127621,0.072360,0.029012,8.001151,111.028692,2790.856734


In [206]:
area['area_km_2'] = area['lat']*111*area['lon']*111.321*math.pi

In [208]:
area.sort_values(by = 'area_km_2')

,lon,lat,ellipse,lat_km,lon_km,area_km,area_km_2
county_name,,,,,,,
Wibaux County,0.081696,0.069986,0.017963,7.738686,111.047483,2699.764095,221.955992
Treasure County,0.138295,0.045461,0.019751,5.026839,111.204986,1756.180464,244.061316
Golden Valley County,0.162618,0.044875,0.022925,4.961956,111.207935,1733.558661,283.280908
Liberty County,0.127621,0.072360,0.029012,8.001151,111.028692,2790.856734,358.485884
Blaine County,0.258872,0.042086,0.034227,4.653571,111.221430,1626.015602,422.930547
Granite County,0.132964,0.084487,0.035292,9.342056,110.922933,3255.469735,436.087249
Prairie County,0.157117,0.071605,0.035344,7.917660,111.034737,2761.885144,436.733901
Musselshell County,0.079711,0.155593,0.038963,17.204581,109.975227,5944.137549,481.457014
Fallon County,0.183776,0.074315,0.042906,8.217283,111.012749,2865.833623,530.167520
